# Generate Queries to test RAG System

We put the entire text of the file into context and send it to Gemini-2.5-Flash and ask it to generate a list of queries that we can use to test the RAG system.

In [ ]:
from pydantic import BaseModel
from typing import List
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

class TestCase(BaseModel):
    query: str
    expected_answer: str
    citations: List[str]

class OutputSchema(BaseModel):
    test_cases: List[TestCase]

# Load document context
with open("../texts/adk-docs.txt", "r") as f:
    document_context = f.read()

# Prepare system prompt
system_prompt = """
You are a world-class assistant specializing in generating comprehensive test cases for Retrieval-Augmented Generation (RAG) systems.

Given the full context of a document, your task is to create a diverse set of at least 100 queries to rigorously evaluate the RAG system. Each query should target specific facts, details, or concepts from the context, and once specific queries are exhausted, include broader or inferential questions that still relate to the content.

For each test case, provide:
- `query`: A clear, concise question that could be asked of the RAG system.
- `expected_answer`: The precise answer that should be returned, based strictly on the context.
- `citations`: A list of references (quotes, chapter titles, or locations) from the context that support the answer.

Ensure queries cover a wide range of topics, including factual recall, reasoning, chronology, character analysis, and thematic understanding. Avoid duplication and strive for variety in question types.

Return the output as a JSON object with a list of test cases in the following format:
{{
    "test_cases": [
        {{
            "query": "...",
            "expected_answer": "...",
            "citations": ["..."]
        }},
        ...
    ]
}}

Context:
{context}
"""

# Set up Gemini LLM with LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    thinking_budget=0,
    response_mime_type="application/json",
    response_schema=OutputSchema.schema()
)

prompt = PromptTemplate(
    input_variables=["context"],
    template=system_prompt
)

chain = LLMChain(llm=llm, prompt=prompt)

# Generate the test set
result = chain.run(context=document_context)

result

/var/folders/yy/t6y4brw52tvdr8k9xbwc01d40000gn/T/ipykernel_3158/2284753437.py:61: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response_schema=OutputSchema.schema()
/var/folders/yy/t6y4brw52tvdr8k9xbwc01d40000gn/T/ipykernel_3158/2284753437.py:69: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/var/folders/yy/t6y4brw52tvdr8k9xbwc01d40000gn/T/ipykernel_3158/2284753437.py:72: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(context=document_context)
Key '$defs' is not supported in schema, ignoring


'{\n  "test_cases": [\n    {\n      "citations": [\n        ".github/CODEOWNERS"\n      ],\n      "expected_answer": "The code owners file controls who is tagged for review for any given pull request.",\n      "query": "What is the purpose of the CODEOWNERS file?"\n    },\n    {\n      "citations": [\n        ".github/CODEOWNERS"\n      ],\n      "expected_answer": "The GitHub team `@GoogleCloudPlatform/teams/generative-ai-devrel` is the code owner for all files.",\n      "query": "Who is the code owner for all files in the repository?"\n    },\n    {\n      "citations": [\n        ".github/CODEOWNERS"\n      ],\n      "expected_answer": "GitHub teams are preferred over individuals to make it easier to find a suitable owner when someone is OOO or leaves the team.",\n      "query": "Why are GitHub teams preferred over individuals in the CODEOWNERS file?"\n    },\n    {\n      "citations": [\n        ".github/ISSUE_TEMPLATE/bug_report.md"\n      ],\n      "expected_answer": "The bug repo

In [2]:
import json

parsed = json.loads(result)
print(parsed["test_cases"])

[{'citations': ['.github/CODEOWNERS'], 'expected_answer': 'The code owners file controls who is tagged for review for any given pull request.', 'query': 'What is the purpose of the CODEOWNERS file?'}, {'citations': ['.github/CODEOWNERS'], 'expected_answer': 'The GitHub team `@GoogleCloudPlatform/teams/generative-ai-devrel` is the code owner for all files.', 'query': 'Who is the code owner for all files in the repository?'}, {'citations': ['.github/CODEOWNERS'], 'expected_answer': 'GitHub teams are preferred over individuals to make it easier to find a suitable owner when someone is OOO or leaves the team.', 'query': 'Why are GitHub teams preferred over individuals in the CODEOWNERS file?'}, {'citations': ['.github/ISSUE_TEMPLATE/bug_report.md'], 'expected_answer': "The bug report template includes sections for 'Describe the bug', 'To Reproduce', 'Expected behavior', 'Screenshots', 'Versions', and 'Additional context'.", 'query': 'What sections are included in the bug report issue templ

# Turn into DataFrame

In [3]:
import pandas as pd
df = pd.DataFrame(parsed["test_cases"])
df.head()

,citations,expected_answer,query
0,[.github/CODEOWNERS],The code owners file controls who is tagged fo...,What is the purpose of the CODEOWNERS file?
1,[.github/CODEOWNERS],The GitHub team `@GoogleCloudPlatform/teams/ge...,Who is the code owner for all files in the rep...
2,[.github/CODEOWNERS],GitHub teams are preferred over individuals to...,Why are GitHub teams preferred over individual...
3,[.github/ISSUE_TEMPLATE/bug_report.md],The bug report template includes sections for ...,What sections are included in the bug report i...
4,[.github/ISSUE_TEMPLATE/bug_report.md],"To reproduce a bug, the template suggests step...",What are the suggested steps to reproduce a bu...


In [4]:
# Reorder the columns, query first, then expected_answer, then citations
df = df[["query", "expected_answer", "citations"]]

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("./tests/longADKDocsTestCases.csv", index=False)